Dans ce TP, nous considérons des trajets en vélo partagé (similaire au vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

Charger le fichier stationData.csv dans un DataFrame station_df et le fichier tripData.csv dans un DataFrame trip_df. Pour chaque Dataframe, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [0]:
%fs ls /FileStore/tables/stationData-8.csv

path,name,size,modificationTime
dbfs:/FileStore/tables/stationData-8.csv,stationData-8.csv,5201,1728312920000


Afficher les schémas des 2 DataFrames.

In [0]:
station_df = spark.read.option(
    "header", "true"
).csv(
    "/FileStore/tables/stationData-8.csv"
)
trip_df = spark.read.csv("/FileStore/tables/tripData-8.csv", header=True)

In [0]:
station_df.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dockcount: string (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



Créer une vue pour chaque DataFrame.

In [0]:
station_df.createOrReplaceTempView("station_view")
trip_df.createOrReplaceTempView("trip_view")

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur trip_df directement, l'autre en rédigeant une requête SQL de la vue correspondant au DataFrame tripData.

In [0]:
print(trip_df.count())

354152


In [0]:
res = spark.sql(
    """
    SELECT count(*)
    FROM trip_view
    """
)
res.show()

+--------+
|count(1)|
+--------+
|  354152|
+--------+



Ecrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [0]:
res = spark.sql(
    """
    SELECT count(*)
    FROM trip_view
    WHERE StartTerminal=EndTerminal
    """
)
res.show()

+--------+
|count(1)|
+--------+
|   10276|
+--------+



On souhaite désormais obtenir l’id des stations associées à ces trajets. Ecrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.
<br>Exemple de sortie :
<br>+--------+--------+
<br>|terminal|count(1)|
<br>+--------+--------+
<br>| 60| 850|
<br>| 50| 708|
<br>| 35| 348|
<br>| 76| 320|
<br>| 74| 307|
<br>(La station 60 est la plus concernée par ces trajets cycliques, avec 850 de ces trajets.)

In [0]:
res = spark.sql(
    """
    SELECT StartTerminal as terminal, count(*) as count
    FROM trip_view
    WHERE StartTerminal=EndTerminal
    GROUP BY terminal
    ORDER BY count DESC
    """
)

res.show()

+--------+-----+
|terminal|count|
+--------+-----+
|      60|  850|
|      50|  708|
|      35|  348|
|      76|  320|
|      74|  307|
|      39|  296|
|      61|  280|
|      67|  277|
|      71|  268|
|      70|  260|
|      28|  254|
|      48|  248|
|      54|  230|
|      69|  227|
|      42|  213|
|      73|  200|
|      57|  197|
|      64|  194|
|       3|  189|
|      72|  181|
+--------+-----+
only showing top 20 rows



Dans la requête précédente, nous avons oublié un élément qui nous importe. Nous souhaitons compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.
<br>Exemple de sortie :
<br>+--------+---------+--------+
<br>|terminal|dockcount|count(1)|
<br>+--------+---------+--------+
<br>| 60| 15| 850|
<br>| 50| 23| 708|
<br>| 35| 11| 348|
<br>| 76| 19| 320|
<br>| 74| 23| 307|
<br>Mettre à jour la requête.

In [0]:
res = spark.sql(
    """
    SELECT StartTerminal as terminal, dockcount, count(*) as count
    FROM trip_view, station_view
    WHERE StartTerminal=EndTerminal AND station_id=StartTerminal
    GROUP BY terminal, dockcount
    ORDER BY count DESC
    """
)

res.show()

+--------+---------+-----+
|terminal|dockcount|count|
+--------+---------+-----+
|      60|       15|  850|
|      50|       23|  708|
|      35|       11|  348|
|      76|       19|  320|
|      74|       23|  307|
|      39|       19|  296|
|      61|       27|  280|
|      67|       27|  277|
|      71|       19|  268|
|      70|       19|  260|
|      28|       23|  254|
|      48|       15|  248|
|      54|       15|  230|
|      69|       23|  227|
|      42|       15|  213|
|      73|       15|  200|
|      57|       15|  197|
|      64|       15|  194|
|       3|       15|  189|
|      72|       23|  181|
+--------+---------+-----+
only showing top 20 rows



Rédiger les 2 requêtes précédentes avec le DSL de DataFrame.

In [0]:
from pyspark.sql.functions import col, desc

In [0]:
res = spark.sql(
    """
    SELECT StartTerminal as terminal, dockcount, count(*) as count
    FROM trip_view, station_view
    WHERE StartTerminal=EndTerminal AND station_id=StartTerminal
    GROUP BY terminal, dockcount
    ORDER BY count DESC
    """
)

res.show()

In [0]:
df_out_2 = trip_df.filter(
    col("StartStation") == col("EndStation")
).join(
    station_df,
    trip_df["StartTerminal"] == station_df["station_id"]
).groupBy(
    "StartTerminal", "dockcount"
).count(
).orderBy(
    desc("count")
)

In [0]:
df_out.show()

+-------------+---------+-----+
|StartTerminal|dockcount|count|
+-------------+---------+-----+
|           60|       15|  850|
|           50|       23|  708|
|           35|       11|  348|
|           76|       19|  320|
|           74|       23|  307|
|           39|       19|  296|
|           61|       27|  280|
|           67|       27|  277|
|           71|       19|  268|
|           70|       19|  260|
|           28|       23|  254|
|           48|       15|  248|
|           54|       15|  230|
|           69|       23|  227|
|           42|       15|  213|
|           73|       15|  200|
|           57|       15|  197|
|           64|       15|  194|
|            3|       15|  189|
|           72|       23|  181|
+-------------+---------+-----+
only showing top 20 rows



Observer le plan d’exécution des requêtes.

In [0]:
df_out_2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   Sort [count#1083L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#1083L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=2627]
      +- Project [StartTerminal#293, dockcount#262, count(1)#1082L AS count#1083L]
         +- HashAggregate(keys=[StartTerminal#293, dockcount#262], functions=[finalmerge_count(merge count#1088L) AS count(1)#1082L])
            +- Exchange hashpartitioning(StartTerminal#293, dockcount#262, 200), ENSURE_REQUIREMENTS, [plan_id=2623]
               +- HashAggregate(keys=[StartTerminal#293, dockcount#262], functions=[merge_count(merge count#1088L) AS count#1088L])
                  +- Project [StartTerminal#293, dockcount#262, count#1088L]
                     +- BroadcastHashJoin [StartTerminal#293], [station_id#258], Inner, BuildRight, false, true
                        :- HashAggregate(keys=[StartTerminal#293], functions=[partial_count(1) AS count#1088L])
    

In [0]:
df_out.explain(True)

== Parsed Logical Plan ==
'Sort ['count DESC NULLS LAST], true
+- Aggregate [StartTerminal#293, dockcount#262], [StartTerminal#293, dockcount#262, count(1) AS count#915L]
   +- Join Inner, (StartTerminal#293 = station_id#258)
      :- Filter (StartStation#292 = EndStation#295)
      :  +- Relation [TripID#289,Duration#290,StartDate#291,StartStation#292,StartTerminal#293,EndDate#294,EndStation#295,EndTerminal#296,Bike##297,SubscriberType#298,ZipCode#299] csv
      +- Relation [station_id#258,name#259,lat#260,long#261,dockcount#262,landmark#263,installation#264] csv

== Analyzed Logical Plan ==
StartTerminal: string, dockcount: string, count: bigint
Sort [count#915L DESC NULLS LAST], true
+- Aggregate [StartTerminal#293, dockcount#262], [StartTerminal#293, dockcount#262, count(1) AS count#915L]
   +- Join Inner, (StartTerminal#293 = station_id#258)
      :- Filter (StartStation#292 = EndStation#295)
      :  +- Relation [TripID#289,Duration#290,StartDate#291,StartStation#292,StartTermina